In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.wgan import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(0)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [3]:
DATA_PATH = Path('data/imagenet/ILSVRC/Data/CLS-LOC')
TRAIN_SOURCE_PATH = DATA_PATH/'train'
proj_id = 'bw2color'
TENSORBOARD_PATH = Path('data/tensorboard/WganModel_z')


dpath = DATA_PATH/(proj_id + '_34_D.h5')
gpath = DATA_PATH/(proj_id + '_34_G.h5')
keep_pct=0.1
bs=2
sz=224

c_lr=3e-4
c_lrs = np.array([c_lr/1000,c_lr/100,c_lr])

g_lr=1e-4
g_lrs = np.array([g_lr/10000,g_lr/100,g_lr])

torch.backends.cudnn.benchmark=True

## Training

In [4]:
netG = Unet34(nf_factor=2).cuda()
netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
#load_model(netG, gpath)

netD = FeatureCritic(sz=sz,filter_scale=4).cuda()
netDVis = ModelVisualizationHook(TENSORBOARD_PATH, netD, 'netD')
#load_model(netD, dpath)

In [5]:
md = get_matched_image_model_data(image_size=sz, batch_size=bs, root_data_path=DATA_PATH, train_root_path=TRAIN_SOURCE_PATH, proj_id=proj_id, keep_pct=keep_pct)

In [6]:
trainer = WGANTrainer(netD=netD, netG=netG, md=md, bs=bs, sz=sz, dpath=dpath, gpath=gpath, genloss_fns=[FeatureLoss(multiplier=1e5)])
trainerVis = WganVisualizationHook(TENSORBOARD_PATH, trainer, 'trainer')

In [ ]:
#netG.freeze_to(1)
netD.freeze_to(1)
trainer.train(lrs_critic=c_lrs, lrs_gen=g_lrs, clr_critic=(1,8, 0.9, 0.9), clr_gen=(1,8, 0.9, 0.9), cycle_len=1, epochs=1)

  0%|          | 20/57603 [00:21<15:05:22,  1.06it/s]
WDist -0.571846067905426; RScore 0.8891487717628479; FScore 0.3173027038574219; GAddlLoss 11918.55078125; GCount: 10; GPenalty: 2.9153764247894287; GCost: -0.3098154067993164; ConPenalty: 1.2172021865844727
  0%|          | 40/57603 [00:37<15:02:09,  1.06it/s]
WDist 0.6609331965446472; RScore -0.11703676730394363; FScore 0.5438964366912842; GAddlLoss 9344.1953125; GCount: 20; GPenalty: 0.5355067253112793; GCost: -0.0721341148018837; ConPenalty: 0.9167649149894714
  0%|          | 60/57603 [00:53<14:57:00,  1.07it/s]
WDist -0.303353488445282; RScore 0.2945026457309723; FScore -0.008850844576954842; GAddlLoss 9435.7265625; GCount: 30; GPenalty: 3.1975302696228027; GCost: -0.41082826256752014; ConPenalty: 0.5655816197395325
  0%|          | 80/57603 [01:09<14:57:33,  1.07it/s]
WDist 0.3359224796295166; RScore -0.3410545587539673; FScore -0.005132071673870087; GAddlLoss 9810.01953125; GCount: 40; GPenalty: 0.19019433856010437; GCost: 0.